In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your processed dataset
train_df = pd.read_csv("train_pdg_fol.csv")
test_df = pd.read_csv("test_pdg_fol.csv")



In [18]:
import numpy as np
import torch
import pandas as pd
import time
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== STEP 1: Data Preprocessing ======
# Rename to standardized column names
train_df = train_df.rename(columns={
    "func_cleaned": "function",
    "target": "label",
    "fol_logic": "fol"
})
test_df = test_df.rename(columns={
    "func_cleaned": "function",
    "target": "label"
})

# Drop rows with missing values
train_df = train_df.dropna(subset=["fol", "function", "label"])
test_df = test_df.dropna(subset=["function", "label"])


train_df["label"] = train_df["label"].astype(int)
test_df["label"] = test_df["label"].astype(int)

# Convert to HuggingFace Datasets
train_ds = Dataset.from_pandas(train_df)
test_ds  = Dataset.from_pandas(test_df)

# ====== STEP 2: Tokenizer Setup ======
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Train: use FOL + FUNC
def tokenize_fn_train(batch):
    return tokenizer(
        ["FOL: " + f + " FUNC: " + c for f, c in zip(batch["fol"], batch["function"])],
        truncation=True, padding='max_length', max_length=256
    )

# Test: use FUNC only
def tokenize_fn_test(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["function"]],
        truncation=True, padding='max_length', max_length=256
    )

# Apply tokenizer
train_tok = train_ds.map(tokenize_fn_train, batched=True)
test_tok  = test_ds.map(tokenize_fn_test, batched=True)

# ====== STEP 3: Prepare datasets ======
train_tok = train_tok.rename_column('label', 'labels')
test_tok  = test_tok.rename_column('label', 'labels')
train_tok.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_tok.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# ====== STEP 4: Load Model ======
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.config.problem_type = "single_label_classification"
model.config.id2label = {0: "safe", 1: "vuln"}
model.config.label2id = {"safe": 0, "vuln": 1}

# ====== STEP 5: Metrics ======
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }

# ====== STEP 6: Training Setup ======
data_collator = DataCollatorWithPadding(tokenizer)
args = TrainingArguments(
    output_dir='./codebert_trainFOL_testFUNC',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    logging_steps=50
)

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

# ====== STEP 7: Train ======
start_train = time.time()
trainer.train()
end_train = time.time()
train_runtime = end_train - start_train
print(f"\n=== Training completed in {train_runtime:.2f} seconds ({train_runtime / 60:.2f} minutes) ===")

# ====== STEP 8: Evaluate ======
start_eval = time.time()
metrics = trainer.evaluate()
end_eval = time.time()
eval_runtime = end_eval - start_eval
print(f"\n=== Evaluation completed in {eval_runtime:.2f} seconds ({eval_runtime / 60:.2f} minutes) ===")

print("\n=== Hold‐out Metrics (Trained with FOL + FUNC, Tested with FUNC only) ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")

# Include runtime in printed metrics
print(f"Train Time (s): {train_runtime:.2f}")
print(f"Eval Time (s):  {eval_runtime:.2f}")

# ====== STEP 9: Save Model ======
save_dir = './codebert_trainFOL_testFUNC_saved'
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"\nModel and tokenizer saved to {save_dir}")

Map: 100%|██████████| 4210/4210 [00:00<00:00, 5321.56 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/shaon/.virtualenvs/PythonProject2/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELIS

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Specificity,Fpr,F1,Mcc,Kappa,Mse,Mae,Auc
1,0.692900,0.682638,0.572447,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.427553,0.427553,0.533904
2,0.681700,0.683794,0.572447,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.427553,0.427553,0.528855



=== Training completed in 555.70 seconds (9.26 minutes) ===



=== Evaluation completed in 26.33 seconds (0.44 minutes) ===

=== Hold‐out Metrics (Trained with FOL + FUNC, Tested with FUNC only) ===
loss        : 0.6826
accuracy    : 0.5724
precision   : 0.0000
recall      : 0.0000
specificity : 1.0000
fpr         : 0.0000
f1          : 0.0000
mcc         : 0.0000
kappa       : 0.0000
mse         : 0.4276
mae         : 0.4276
auc         : 0.5339
runtime     : 26.3286
samples_per_second: 159.9020
steps_per_second: 20.0160
Train Time (s): 555.70
Eval Time (s):  26.33

Model and tokenizer saved to ./codebert_trainFOL_testFUNC_saved


In [3]:
import numpy as np
import torch
import pandas as pd
import time
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== STEP 1: Data Preprocessing ======
# Rename to standardized column names
train_df = train_df.rename(columns={
    "func_cleaned": "function",
    "target": "label",
    "fol_logic": "fol"
})
test_df = test_df.rename(columns={
    "func_cleaned": "function",
    "target": "label"
})

# Drop rows with missing values
train_df = train_df.dropna(subset=["fol", "function", "label"])
test_df = test_df.dropna(subset=["function", "label"])


train_df["label"] = train_df["label"].astype(int)
test_df["label"] = test_df["label"].astype(int)

# Convert to HuggingFace Datasets
train_ds = Dataset.from_pandas(train_df)
test_ds  = Dataset.from_pandas(test_df)

# ====== STEP 2: Tokenizer Setup ======
model_name = "microsoft/graphcodebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Train: use FOL + FUNC
def tokenize_fn_train(batch):
    return tokenizer(
        ["FOL: " + f + " FUNC: " + c for f, c in zip(batch["fol"], batch["function"])],
        truncation=True, padding='max_length', max_length=256
    )

# Test: use FUNC only
def tokenize_fn_test(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["function"]],
        truncation=True, padding='max_length', max_length=256
    )

# Apply tokenizer
train_tok = train_ds.map(tokenize_fn_train, batched=True)
test_tok  = test_ds.map(tokenize_fn_test, batched=True)

# ====== STEP 3: Prepare datasets ======
train_tok = train_tok.rename_column('label', 'labels')
test_tok  = test_tok.rename_column('label', 'labels')
train_tok.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_tok.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# ====== STEP 4: Load Model ======
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
model.config.problem_type = "single_label_classification"
model.config.id2label = {0: "safe", 1: "vuln"}
model.config.label2id = {"safe": 0, "vuln": 1}

# ====== STEP 5: Metrics ======
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }

# ====== STEP 6: Training Setup ======
data_collator = DataCollatorWithPadding(tokenizer)
args = TrainingArguments(
    output_dir='./codebert_trainFOL_testFUNC',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    logging_steps=50
)

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_tok,
    eval_dataset=test_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

# ====== STEP 7: Train ======
start_train = time.time()
trainer.train()
end_train = time.time()
train_runtime = end_train - start_train
print(f"\n=== Training completed in {train_runtime:.2f} seconds ({train_runtime / 60:.2f} minutes) ===")

# ====== STEP 8: Evaluate ======
start_eval = time.time()
metrics = trainer.evaluate()
end_eval = time.time()
eval_runtime = end_eval - start_eval
print(f"\n=== Evaluation completed in {eval_runtime:.2f} seconds ({eval_runtime / 60:.2f} minutes) ===")

print("\n=== Hold‐out Metrics (Trained with FOL + FUNC, Tested with FUNC only) ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")

# Include runtime in printed metrics
print(f"Train Time (s): {train_runtime:.2f}")
print(f"Eval Time (s):  {eval_runtime:.2f}")

# ====== STEP 9: Save Model ======
save_dir = './graphcodebert_trainFOL_testFUNC_saved'
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"\nModel and tokenizer saved to {save_dir}")

Map: 100%|██████████| 4210/4210 [00:01<00:00, 3653.56 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/shaon/.virtualenvs/PythonProject2/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARAL

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Specificity,Fpr,F1,Mcc,Kappa,Mse,Mae,Auc
1,0.686600,0.672901,0.578147,0.639535,0.030556,0.987137,0.012863,0.058324,0.061876,0.020117,0.421853,0.421853,0.597101
2,0.653800,0.660138,0.600713,0.553172,0.343889,0.792531,0.207469,0.424118,0.152777,0.143280,0.399287,0.399287,0.628853
3,0.559800,0.713293,0.598100,0.524324,0.646667,0.561826,0.438174,0.579104,0.206601,0.202512,0.401900,0.401900,0.646500
4,0.509900,0.758144,0.595249,0.523692,0.589444,0.599585,0.400415,0.554626,0.187167,0.186073,0.404751,0.404751,0.645234



=== Training completed in 3588.59 seconds (59.81 minutes) ===



=== Evaluation completed in 27.45 seconds (0.46 minutes) ===

=== Hold‐out Metrics (Trained with FOL + FUNC, Tested with FUNC only) ===
loss        : 0.7133
accuracy    : 0.5981
precision   : 0.5243
recall      : 0.6467
specificity : 0.5618
fpr         : 0.4382
f1          : 0.5791
mcc         : 0.2066
kappa       : 0.2025
mse         : 0.4019
mae         : 0.4019
auc         : 0.6465
runtime     : 27.4372
samples_per_second: 153.4410
steps_per_second: 19.2080
Train Time (s): 3588.59
Eval Time (s):  27.45

Model and tokenizer saved to ./graphcodebert_trainFOL_testFUNC_saved


bigvul_test

In [4]:
import pandas as pd
gbig_test_df = pd.read_csv("big_vultest.csv")
gbig_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1170 entries, 0 to 1169
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  1170 non-null   object
 1   input        1170 non-null   object
 2   output       1170 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 27.5+ KB


In [7]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './graphcodebert_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
gbig_test_df = gbig_test_df.rename(columns={"input": "func", "output": "label"})
gbig_test_df = gbig_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
gbig_test_ds = Dataset.from_pandas(gbig_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["func"]],
        truncation=True, padding='max_length', max_length=256
    )

big_test_tok = gbig_test_ds.map(tokenize_fn, batched=True)
big_test_tok = big_test_tok.rename_column("label", "labels")
big_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=big_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on BigVul Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map: 100%|██████████| 1170/1170 [00:00<00:00, 8878.38 examples/s]
/tmp/ipykernel_5884/2969780316.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on BigVul Dataset ===
loss        : 0.7886
model_preparation_time: 0.0018
accuracy    : 0.5171
precision   : 0.5174
recall      : 0.5094
specificity : 0.5248
fpr         : 0.4752
f1          : 0.5134
mcc         : 0.0342
kappa       : 0.0342
mse         : 0.4829
mae         : 0.4829
auc         : 0.5276
runtime     : 8.5144
samples_per_second: 137.4140
steps_per_second: 17.2650


codebert

In [19]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './codebert_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
gbig_test_df = gbig_test_df.rename(columns={"input": "func", "output": "label"})
gbig_test_df = gbig_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
gbig_test_ds = Dataset.from_pandas(gbig_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["func"]],
        truncation=True, padding='max_length', max_length=256
    )

big_test_tok = gbig_test_ds.map(tokenize_fn, batched=True)
big_test_tok = big_test_tok.rename_column("label", "labels")
big_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=big_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on BigVul Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")

Map: 100%|██████████| 1170/1170 [00:00<00:00, 8902.29 examples/s]
/tmp/ipykernel_5884/2700946773.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on BigVul Dataset ===
loss        : 0.7050
model_preparation_time: 0.0018
accuracy    : 0.5000
precision   : 0.0000
recall      : 0.0000
specificity : 1.0000
fpr         : 0.0000
f1          : 0.0000
mcc         : 0.0000
kappa       : 0.0000
mse         : 0.5000
mae         : 0.5000
auc         : 0.5765
runtime     : 7.6790
samples_per_second: 152.3630
steps_per_second: 19.1430


diverse

In [8]:
import pandas as pd
gdiv_test_df = pd.read_csv("diverse_test.csv")
gdiv_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1532 entries, 0 to 1531
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   code_snip  1532 non-null   object
 1   output     1532 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 24.1+ KB


In [9]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './graphcodebert_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
gdiv_test_df = gdiv_test_df.rename(columns={"code_snip": "func", "output": "label"})
gdiv_test_df= gdiv_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
gdiv_test_ds = Dataset.from_pandas(gdiv_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["func"]],
        truncation=True, padding='max_length', max_length=256
    )

div_test_tok = gdiv_test_ds.map(tokenize_fn, batched=True)
div_test_tok = div_test_tok.rename_column("label", "labels")
div_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=div_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on Diverse Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map: 100%|██████████| 1532/1532 [00:00<00:00, 9524.06 examples/s]
/tmp/ipykernel_5884/3833255184.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on Diverse Dataset ===
loss        : 0.7720
model_preparation_time: 0.0019
accuracy    : 0.5437
precision   : 0.5414
recall      : 0.5718
specificity : 0.5157
fpr         : 0.4843
f1          : 0.5562
mcc         : 0.0876
kappa       : 0.0875
mse         : 0.4563
mae         : 0.4563
auc         : 0.5646
runtime     : 11.3230
samples_per_second: 135.3000
steps_per_second: 16.9570


Juliet

In [10]:
import pandas as pd
juliet_test_df = pd.read_csv("djuliet_test.csv")
juliet_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3152 entries, 0 to 3151
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   code_snip  3152 non-null   object
 1   output     3152 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 49.4+ KB


In [11]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './graphcodebert_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
juliet_test_df = juliet_test_df.rename(columns={"code_snip": "func", "output": "label"})
juliet_test_df = juliet_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
juliet_test_ds = Dataset.from_pandas(juliet_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["func"]],
        truncation=True, padding='max_length', max_length=256
    )

juliet_test_tok = juliet_test_ds.map(tokenize_fn, batched=True)
juliet_test_tok = juliet_test_tok.rename_column("label", "labels")
juliet_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=juliet_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on Juliet Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map: 100%|██████████| 3152/3152 [00:00<00:00, 10490.84 examples/s]
/tmp/ipykernel_5884/2579193972.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on Juliet Dataset ===
loss        : 0.7183
model_preparation_time: 0.0039
accuracy    : 0.5470
precision   : 0.5510
recall      : 0.5076
specificity : 0.5863
fpr         : 0.4137
f1          : 0.5284
mcc         : 0.0942
kappa       : 0.0939
mse         : 0.4530
mae         : 0.4530
auc         : 0.5641
runtime     : 24.7784
samples_per_second: 127.2070
steps_per_second: 15.9010


Reveal

In [12]:
import pandas as pd
reveal_test_df = pd.read_csv("Reveal_vultest.csv")
reveal_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2028 entries, 0 to 2027
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  2028 non-null   object
 1   input        2028 non-null   object
 2   output       2028 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 47.7+ KB


In [13]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './graphcodebert_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame
reveal_test_df = pd.read_csv("Reveal_vultest.csv")

# Standardize column names and drop missing values
reveal_test_df = reveal_test_df.rename(columns={"input": "func", "output": "label"})
reveal_test_df = reveal_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
reveal_test_ds = Dataset.from_pandas(reveal_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["func"]],
        truncation=True, padding='max_length', max_length=256
    )

reveal_test_tok = reveal_test_ds.map(tokenize_fn, batched=True)
reveal_test_tok = reveal_test_tok.rename_column("label", "labels")
reveal_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=reveal_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on Reveal Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map: 100%|██████████| 2028/2028 [00:00<00:00, 7563.58 examples/s]
/tmp/ipykernel_5884/2308563442.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on Reveal Dataset ===
loss        : 0.8036
model_preparation_time: 0.0022
accuracy    : 0.5242
precision   : 0.5250
recall      : 0.5079
specificity : 0.5404
fpr         : 0.4596
f1          : 0.5163
mcc         : 0.0483
kappa       : 0.0483
mse         : 0.4758
mae         : 0.4758
auc         : 0.5337
runtime     : 13.3345
samples_per_second: 152.0860
steps_per_second: 19.0480


MixVUl

In [14]:
import pandas as pd
mix_test_df = pd.read_csv("mix_test_vultest.csv")
mix_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2864 entries, 0 to 2863
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  2864 non-null   object
 1   input        2864 non-null   object
 2   output       2864 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 67.2+ KB


In [15]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './graphcodebert_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======


# Standardize column names and drop missing values
mix_test_df = mix_test_df.rename(columns={"input": "func", "output": "label"})
mix_test_df = mix_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
mix_test_ds = Dataset.from_pandas(mix_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["func"]],
        truncation=True, padding='max_length', max_length=256
    )

mix_test_tok = mix_test_ds.map(tokenize_fn, batched=True)
mix_test_tok = mix_test_tok.rename_column("label", "labels")
mix_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=mix_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on mix Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map: 100%|██████████| 2864/2864 [00:00<00:00, 5310.75 examples/s]
/tmp/ipykernel_5884/3312201838.py:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on mix Dataset ===
loss        : 0.7283
model_preparation_time: 0.0021
accuracy    : 0.5744
precision   : 0.5688
recall      : 0.6145
specificity : 0.5342
fpr         : 0.4658
f1          : 0.5908
mcc         : 0.1492
kappa       : 0.1487
mse         : 0.4256
mae         : 0.4256
auc         : 0.6111
runtime     : 17.3509
samples_per_second: 165.0640
steps_per_second: 20.6330


CVEfix

In [16]:
import pandas as pd

fun_test_df = pd.read_json("test_512.json")
fun_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4216 entries, 0 to 4215
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  4216 non-null   object
 1   input        4216 non-null   object
 2   output       4216 non-null   int64 
 3   idx          4216 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 131.9+ KB


In [17]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './graphcodebert_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
fun_test_df = fun_test_df.rename(columns={"input": "func", "output": "label"})
fun_test_df = fun_test_df.dropna(subset=["func", "label"])


# Convert to HuggingFace Dataset
fun_test_ds = Dataset.from_pandas(fun_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["func"]],
        truncation=True, padding='max_length', max_length=256
    )

fun_test_tok = fun_test_ds.map(tokenize_fn, batched=True)
fun_test_tok = fun_test_tok.rename_column("label", "labels")
fun_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=fun_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on VULLM fol without Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


Map: 100%|██████████| 4216/4216 [00:00<00:00, 8370.31 examples/s]
/tmp/ipykernel_5884/529823862.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on VULLM fol without Dataset ===
loss        : 0.8340
model_preparation_time: 0.0018
accuracy    : 0.5021
precision   : 0.5018
recall      : 0.5840
specificity : 0.4203
fpr         : 0.5797
f1          : 0.5398
mcc         : 0.0043
kappa       : 0.0043
mse         : 0.4979
mae         : 0.4979
auc         : 0.5032
runtime     : 25.6919
samples_per_second: 164.0980
steps_per_second: 20.5120


mix vul

In [1]:
import pandas as pd
mix_test_df = pd.read_csv("mixvul_test_fol.csv")
mix_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2864 entries, 0 to 2863
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  2864 non-null   object
 1   input        2864 non-null   object
 2   output       2864 non-null   int64 
 3   fol_logic    2864 non-null   object
dtypes: int64(1), object(3)
memory usage: 89.6+ KB


In [2]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './graphcodebert_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
mix_test_df = mix_test_df.rename(columns={"input": "mixc", "output": "label"})
mix_test_df = mix_test_df.dropna(subset=["mixc", "label"])


# Convert to HuggingFace Dataset
mix_test_ds = Dataset.from_pandas(mix_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["mixc"]],
        truncation=True, padding='max_length', max_length=256
    )

mix_test_tok = mix_test_ds.map(tokenize_fn, batched=True)
mix_test_tok = mix_test_tok.rename_column("label", "labels")
mix_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=mix_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on VULLM fol without Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


/home/shaon/.virtualenvs/PythonProject2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 2864/2864 [00:00<00:00, 7046.32 examples/s]
/tmp/ipykernel_1372981/793645606.py:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on VULLM fol without Dataset ===
loss        : 0.7283
model_preparation_time: 0.0022
accuracy    : 0.5744
precision   : 0.5688
recall      : 0.6145
specificity : 0.5342
fpr         : 0.4658
f1          : 0.5908
mcc         : 0.1492
kappa       : 0.1487
mse         : 0.4256
mae         : 0.4256
auc         : 0.6111
runtime     : 19.3377
samples_per_second: 148.1050
steps_per_second: 18.5130


In [1]:
import pandas as pd
CV_test_df = pd.read_csv("mixvul_test_fol.csv")
CV_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2864 entries, 0 to 2863
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  2864 non-null   object
 1   input        2864 non-null   object
 2   output       2864 non-null   int64 
 3   fol_logic    2864 non-null   object
dtypes: int64(1), object(3)
memory usage: 89.6+ KB


In [2]:
import numpy as np
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, matthews_corrcoef,
    cohen_kappa_score, mean_squared_error,
    mean_absolute_error, roc_auc_score
)

# ====== LOAD SAVED MODEL ======
saved_dir = './graphcodebert_trainFOL_testFUNC_saved'
model = AutoModelForSequenceClassification.from_pretrained(saved_dir)
tokenizer = AutoTokenizer.from_pretrained(saved_dir)
data_collator = DataCollatorWithPadding(tokenizer)

# ====== LOAD AND PROCESS NEW TEST DATA ======
# Replace with your actual new test DataFrame


# Standardize column names and drop missing values
CV_test_df = CV_test_df.rename(columns={"input": "func", "output": "label"})
CV_test_df = CV_test_df.dropna(subset=["func", "label"])

SEP = " // LOGIC: "
CV_test_df['model_input'] =CV_test_df['func'] + SEP +  CV_test_df['fol_logic']
# Convert to HuggingFace Dataset
CV_test_ds = Dataset.from_pandas(CV_test_df)

# Tokenization (FUNC only, no FOL)
def tokenize_fn(batch):
    return tokenizer(
        ["FUNC: " + c for c in batch["fol_logic"]] ,
        truncation=True, padding='max_length', max_length=256
    )

CV_test_tok = CV_test_ds.map(tokenize_fn, batched=True)
CV_test_tok = CV_test_tok.rename_column("label", "labels")
CV_test_tok.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:, 1]
    try:
        auc = roc_auc_score(labels, probs)
    except ValueError:
        auc = float('nan')
    return {
        "accuracy":    accuracy_score(labels, preds),
        "precision":   precision_score(labels, preds, zero_division=0),
        "recall":      recall_score(labels, preds, zero_division=0),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0,
        "fpr":         fp / (fp + tn) if (fp + tn) > 0 else 0,
        "f1":          f1_score(labels, preds, zero_division=0),
        "mcc":         matthews_corrcoef(labels, preds),
        "kappa":       cohen_kappa_score(labels, preds),
        "mse":         mean_squared_error(labels, preds),
        "mae":         mean_absolute_error(labels, preds),
        "auc":         auc
    }


trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate(eval_dataset=CV_test_tok)

# ====== PRINT METRICS ======
print("\n=== Evaluation on RVl Dataset ===")
for key, value in metrics.items():
    if key.startswith("eval_"):
        print(f"{key[5:]:<12}: {value:.4f}")


/home/shaon/.virtualenvs/PythonProject2/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 2864/2864 [00:01<00:00, 1628.07 examples/s]
/tmp/ipykernel_25672/1219205585.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



=== Evaluation on RVl Dataset ===
loss        : 0.7370
model_preparation_time: 0.0019
accuracy    : 0.5660
precision   : 0.5735
recall      : 0.5147
specificity : 0.6173
fpr         : 0.3827
f1          : 0.5425
mcc         : 0.1327
kappa       : 0.1320
mse         : 0.4340
mae         : 0.4340
auc         : 0.6069
runtime     : 19.0799
samples_per_second: 150.1050
steps_per_second: 18.7630
